You are using Copilot for free<br>
Responses are limited to **2,000** `code completions` and **50** `chat messages` *per month*.

In [ ]:
# vector embeddings for text using OpenAI's API and langchain framework
import os
from langchain.embeddings import OpenAIEmbeddings
from langchain.vectorstores import Chroma
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.document_loaders import TextLoader
from langchain.llms import OpenAI
from langchain.chains import RetrievalQA

In [ ]:
#

In [ ]:
# # common package installations
# pip install --upgrade --quiet langchain_community
# pip install --upgrade --quiet PyPDF2
# # pip install --upgrade --quiet langchain-google-genai
# # pip install --upgrade --quiet google-ai-generativelanguage==0.6.15
# # pip install --upgrade --quiet langchain_huggingface
# pip install --upgrade --quiet langchain-chroma
# pip install --upgrade --quiet pypdf
# pip install --quiet google-ai-generativelanguage==0.6.15
# pip install --upgrade --quiet google-generativeai

In [ ]:
# common imports
import getpass
import os, csv

from langchain_community.document_loaders import TextLoader
from langchain_community.vectorstores import Chroma
from langchain_community.document_loaders import PyPDFLoader
from langchain_google_genai import ChatGoogleGenerativeAI
from langchain_google_genai import GoogleGenerativeAIEmbeddings
from langchain_core.prompts import ChatPromptTemplate
from langchain.schema.runnable import RunnablePassthrough
from langchain_core.output_parsers import StrOutputParser

In [2]:
import os, csv

In [ ]:
# Document Summerization
from langchain.prompts import PromptTemplate
from langchain.chains.combine_documents.stuff import StuffDocumentsChain
import textwrap

In [ ]:
# Load documents
file_path = '/content/Corporate Governance in India.pdf'
loader = PyPDFLoader(file_path)
documents = loader.load()
print(f"Loaded {len(documents)} documents from the folder.")

In [ ]:
# read the confige file
config_file_path = r'./configs.txt'
with open(config_file_path,'r') as config_file:
    for row in csv.DictReader(config_file, delimiter='='):
        vars()[row['variable_name']] = row['variable_value']

In [ ]:
if "GOOGLE_API_KEY" not in os.environ:
    os.environ["GOOGLE_API_KEY"] = GEMINI_API_KEY

In [ ]:
# Create LLM
llm = ChatGoogleGenerativeAI(model="gemini-2.5-flash")

In [ ]:
# test LLM
llm_response = llm.invoke("Tell me a joke about economics")
print(llm_response.content)

Here are a few economics jokes for you:

1.  **How many economists does it take to change a lightbulb?**
    One, but on the one hand, the old bulb might still have some utility, and on the other hand, the cost of replacement might not justify the marginal benefit.

2.  **A physicist, a chemist, and an economist are stranded on a desert island with a can of food.**
    The physicist says, "We can calculate the force needed to open it."
    The chemist says, "We can analyze its composition to see if it's safe to eat."
    The economist says, "Assume we have a can opener."

3.  **What's the difference between an economist and a prophet?**
    A prophet can tell you what will happen. An economist can tell you why it didn't.

4.  **Why did the economist break up with the statistician?**
    He just couldn't deal with all the "on the one hand, on the other hand" arguments.

Hope those give you a laugh!

In [ ]:
# RAG development
# Create embeding model
embeddings = GoogleGenerativeAIEmbeddings(model="models/gemini-embedding-001")

In [ ]:
# Create Chroma vector store from documents
vectorstore = Chroma.from_documents(
    documents=documents,
    embedding=embeddings,
    collection_name="my_document_collection",
    persist_directory="/content/chroma_db"
)

In [ ]:
retriever = vectorstore.as_retriever(search_kwargs={"k": 2})
retriever.invoke("What this corporate gevernance?")

[Document(metadata={'author': 'mk sridhar', 'creationdate': '2022-01-28T06:21:12+00:00', 'page_label': '16', 'total_pages': 64, 'source': '/content/Corporate Governance in India.pdf', 'moddate': '2022-01-28T06:21:12+00:00', 'creator': 'Microsoft® Word 2016', 'producer': 'www.ilovepdf.com', 'page': 15}, page_content='12 \n \n \n \n \n1. The Principles of Governance \n \n \nThe OECD Principles of Corporate Governance, endorsed by OECD  Ministries have since become an \ninternational benchmark for policy makers, investors, corporations and other stakeholders \nworldwide. They have advanced the corporate governance agenda and provided specific guidance for \nlegislative and regulatory initiatives in both OECD and non-OECD countries. \n \nThese principles are broadly: \n \n \nSource:http://www.oecd.org/daf/ca/Corproate-Governance-Principles-ENG.pdf'),
 Document(metadata={'page': 20, 'author': 'mk sridhar', 'creationdate': '2022-01-28T06:21:12+00:00', 'total_pages': 64, 'creator': 'Microsoft® Word 2016', 'producer': 'www.ilovepdf.com', 'moddate': '2022-01-28T06:21:12+00:00', 'source': '/content/Corporate Governance in India.pdf', 'page_label': '21'}, page_content='17 \n \n \nThe core concepts Corporate Governance can be summarised as: \n \na. Transparency \n \nTransparency leads to sufficient reporting without undermining the interest of the \nclient. This needs the clarification of strategies and actions of the company for those \nto whom it is accountable. The Financials are prepared in accordance with IFRS, \nregistry filings are up-to-date, high quality Annual Reports are published,  Web based \ndisclosure in place and timely accurate disclosures of all material matters are made. \n \nb. Rule of law \n \nUnderstand and abide by the various Rule of Law enacted. Anticipate future \nrequirements and prepare well in time to ensure adherence. \n \nc. Participation \n \nEducate and involve all stakeholders in a fair manner while conducting the business. \n \nd. Responsiveness \n \nRespond in a timely, transparent and fair manner \n \ne. Responsibility. \n \nManagement Board is liable to shareholders and shareholders are liable to the \nManagement Board. Responsibility gives success impetus. \n \nf. Equity \n \nBeing fair and practice equity. \n \ng. Ethics')]

In [ ]:
template = """Answer the question based only on the following context:
{context}

Question: {question}

Answer: """
prompt = ChatPromptTemplate.from_template(template)

In [ ]:
rag_chain = (
    {"context": retriever, "question": RunnablePassthrough()} | prompt
)
response = rag_chain.invoke("What this corporate gevernance?")
response

ChatPromptValue(messages=[HumanMessage(content="Answer the question based only on the following context:\n[Document(metadata={'page_label': '16', 'creator': 'Microsoft® Word 2016', 'author': 'mk sridhar', 'page': 15, 'producer': 'www.ilovepdf.com', 'total_pages': 64, 'moddate': '2022-01-28T06:21:12+00:00', 'creationdate': '2022-01-28T06:21:12+00:00', 'source': '/content/Corporate Governance in India.pdf'}, page_content='12 \\n \\n \\n \\n \\n1. The Principles of Governance \\n \\n \\nThe OECD Principles of Corporate Governance, endorsed by OECD  Ministries have since become an \\ninternational benchmark for policy makers, investors, corporations and other stakeholders \\nworldwide. They have advanced the corporate governance agenda and provided specific guidance for \\nlegislative and regulatory initiatives in both OECD and non-OECD countries. \\n \\nThese principles are broadly: \\n \\n \\nSource:http://www.oecd.org/daf/ca/Corproate-Governance-Principles-ENG.pdf'), Document(metadata={'creator': 'Microsoft® Word 2016', 'moddate': '2022-01-28T06:21:12+00:00', 'source': '/content/Corporate Governance in India.pdf', 'author': 'mk sridhar', 'page_label': '21', 'page': 20, 'total_pages': 64, 'producer': 'www.ilovepdf.com', 'creationdate': '2022-01-28T06:21:12+00:00'}, page_content='17 \\n \\n \\nThe core concepts Corporate Governance can be summarised as: \\n \\na. Transparency \\n \\nTransparency leads to sufficient reporting without undermining the interest of the \\nclient. This needs the clarification of strategies and actions of the company for those \\nto whom it is accountable. The Financials are prepared in accordance with IFRS, \\nregistry filings are up-to-date, high quality Annual Reports are published,  Web based \\ndisclosure in place and timely accurate disclosures of all material matters are made. \\n \\nb. Rule of law \\n \\nUnderstand and abide by the various Rule of Law enacted. Anticipate future \\nrequirements and prepare well in time to ensure adherence. \\n \\nc. Participation \\n \\nEducate and involve all stakeholders in a fair manner while conducting the business. \\n \\nd. Responsiveness \\n \\nRespond in a timely, transparent and fair manner \\n \\ne. Responsibility. \\n \\nManagement Board is liable to shareholders and shareholders are liable to the \\nManagement Board. Responsibility gives success impetus. \\n \\nf. Equity \\n \\nBeing fair and practice equity. \\n \\ng. Ethics')]\n\nQuestion: What this corporate gevernance?\n\nAnswer: ", additional_kwargs={}, response_metadata={})])

In [ ]:
def docs2str(docs):
    return "\n\n".join(doc.page_content for doc in docs)

In [ ]:
rag_chain = (
    {"context": retriever | docs2str, "question": RunnablePassthrough()}
    | prompt
    | llm
    | StrOutputParser()
)
question = "What this corporate gevernance?"
response = rag_chain.invoke(question)
print(response)

Based on the provided context, the core concepts of Corporate Governance can be summarised as:

*   **Transparency:** Leads to sufficient reporting without undermining the client's interest, clarifying strategies and actions, with financials prepared according to IFRS, up-to-date registry filings, high-quality Annual Reports, web-based disclosure, and timely accurate disclosures of all material matters.
*   **Rule of law:** Understanding and abiding by enacted rules, anticipating future requirements, and preparing in time for adherence.
*   **Participation:** Educating and involving all stakeholders fairly.
*   **Responsiveness:** Responding in a timely, transparent, and fair manner.
*   **Responsibility:** Management Board is liable to shareholders and vice-versa, which gives success impetus.
*   **Equity:** Being fair and practicing equity.
*   **Ethics**

Additionally, the OECD Principles of Corporate Governance are described as an international benchmark for policy makers, investors, corporations, and other stakeholders worldwide, advancing the corporate governance agenda and providing specific guidance for legislative and regulatory initiatives.

In [ ]:
question = "How to achieve corporate gevernance in corporations?"
response = rag_chain.invoke(question)
print(response)

Based on the provided context, corporate governance can be achieved by adhering to the following core concepts:

*   **Transparency:**
    *   Ensuring sufficient reporting without undermining the interest of the client.
    *   Clarifying strategies and actions of the company for those to whom it is accountable.
    *   Preparing Financials in accordance with IFRS.
    *   Keeping registry filings up-to-date.
    *   Publishing high quality Annual Reports.
    *   Having web-based disclosure in place.
    *   Making timely accurate disclosures of all material matters.
*   **Rule of law:**
    *   Understanding and abiding by the various enacted Rule of Law.
    *   Anticipating future requirements and preparing well in time to ensure adherence.
*   **Participation:**
    *   Educating and involving all stakeholders in a fair manner while conducting the business.
*   **Responsiveness:**
    *   Responding in a timely, transparent, and fair manner.
*   **Responsibility:**
    *   Ensuring the Management Board is liable to shareholders and shareholders are liable to the Management Board.
*   **Equity:**
    *   Being fair and practicing equity.
*   **Ethics** (The text lists this as a core concept but does not provide specific details on how to achieve it).

In [ ]:
# Document Summerization 
prompt_template = """Write a concise summary of the following:
"{text}"
CONCISE SUMMARY:"""
prompt = PromptTemplate.from_template(prompt_template)

In [ ]:
llm_chain = LLMChain(llm=llm, prompt=prompt)
# llm_chain = (prompt | llm)

In [ ]:
stuff_chain = StuffDocumentsChain(
    llm_chain=llm_chain,
    document_variable_name="text"
  )

In [ ]:
output_summary = stuff_chain.run(documents)

In [ ]:
wrapped_text = textwrap.fill(output_summary,
                             width=100,
                             break_long_words=False,
                             replace_whitespace=False)
print(wrapped_text)

This e-book, published by the Southern India Regional Council of The Institute of Chartered
Accountants of India and authored by CA. M K Sridhar, provides a comprehensive overview of Corporate
Governance (CG) in India.

It defines CG as a framework of internal controls, policies, and
procedures ensuring transparency, integrity, ethics, and honesty in a company's operations and
stakeholder dealings. The book traces CG's evolution in India since 1998, shaped by regulatory
bodies like the Ministry of Corporate Affairs (MCA) and SEBI, and various committee recommendations
(e.g., Kumaramangalam Birla, N.R. Narayana Murthy). It details the current legal framework,
primarily the Companies Act, 2013, and SEBI regulations.

Core CG principles discussed include the
OECD guidelines, the "4 Pillars" (vision, stakeholder engagement, compliance/management,
development/succession), and the "4 P's" (people, purpose, process, performance), emphasizing
concepts like transparency, accountability, ethics, and sustainability. It outlines the corporate
governance structure, the fiduciary roles of the Board of Directors and management, and the
importance of IT governance.

A significant portion covers Environmental, Social, and Governance
(ESG) reporting, highlighting the transition from Business Responsibility Reporting (BRR) to the
mandatory Business Responsibility and Sustainability Report (BRSR) for top listed entities, based on
India's National Guidelines on Responsible Business Conduct (NGRBC). The e-book also introduces the
Corporate Governance Scorecard for evaluating Indian companies, discusses Climate-Related Financial
Disclosures (TCFD recommendations), and provides examples of past corporate governance failures.

In [ ]:
prompt_template2 = """Write a concise summary and with important concepts of the following:
"{text}"
"""
prompt2 = PromptTemplate.from_template(prompt_template2)

In [ ]:
llm_chain2 = LLMChain(llm=llm, prompt=prompt2)

In [ ]:
stuff_chain2 = StuffDocumentsChain(
    llm_chain=llm_chain2,
    document_variable_name="text"
  )

In [ ]:
output_summary2 = stuff_chain2.run(documents)

In [ ]:
wrapped_text2 = textwrap.fill(output_summary2,
                             width=100,
                             break_long_words=False,
                             replace_whitespace=False)
print(wrapped_text2)

This e-book by CA. M K Sridhar, published by the Southern India Regional Council of The Institute of
Chartered Accountants of India (ICAI), provides a comprehensive overview of **Corporate Governance
(CG) in India**.

---

### Concise Summary:

The e-book defines Corporate Governance as the
framework of internal controls, policies, and procedures guiding a company's operations and
interactions with various stakeholders, emphasizing principles of transparency, integrity, ethics,
and honesty. It traces the evolution of CG in India since 1998, driven by initiatives from
regulatory bodies like the Ministry of Corporate Affairs (MCA) and SEBI through various committees
(e.g., Kumaramangalam Birla, Naresh Chandra, N R Narayana Murthy) and legislative actions (Companies
Act, SEBI Regulations). The document highlights India's robust CG standards, which are considered
among the best globally. Recent advancements include mandatory Corporate Social Responsibility (CSR)
and the move towards comprehensive Environmental, Social, and Governance (ESG) reporting via the
Business Responsibility and Sustainability Report (BRSR) based on the National Guidelines on
Responsible Business Conduct (NGRBC). The e-book details the core principles, structural elements,
and the importance of measuring CG effectiveness through scorecards and addressing climate-related
financial disclosures (TCFD), aiming to serve as a guide for professionals to foster ethical
corporate practices.

---

### Important Concepts:

1.  **Definition of Corporate Governance:** A
system of internal controls, policies, and procedures forming a company's operational framework,
dealing with stakeholders, and upholding principles of transparency, integrity, ethics, and honesty.
It's considered the "soul of an organization."
2.  **Evolution in India:**
    *   Initiated by
**Confederation of Indian Industries (CII)** in 1995 (Rahul Bajaj Committee), leading to a voluntary
"Desirable Corporate Governance" code (1998).
    *   Further shaped by **SEBI** (Kumaramangalam
Birla Committee, 1999; N R Narayana Murthy Committee, 2002) and **MCA** (Naresh Chandra Committee,
2002).
    *   Key legislative frameworks: **Companies Act (2013)**, **SEBI (Listing Obligations and
Disclosure Requirements) Regulations, 2015**, and the **Listing Agreement**.
3.  **National
Guidelines on Responsible Business Conduct (NGRBC):** Formulated by MCA in 2019, superseding NVGs,
these nine principles guide businesses on ethical conduct, sustainability, employee well-being,
stakeholder engagement, human rights, environmental protection, responsible public advocacy,
inclusive growth, and customer value.
4.  **Core Principles of Governance (OECD & General):**
    *
**OECD Principles:** Effective governance framework, shareholder rights and equitable treatment,
role of institutional investors, stakeholder engagement, disclosure and transparency, and board
responsibilities.
    *   **Fundamental Concepts:** Transparency, Rule of Law, Participation,
Responsiveness, Responsibility, Equity, Ethics, Efficiency & Effectiveness, Independence,
Sustainability, Accountability, Fairness, Trusteeship (of directors), Empowerment, Oversight,
External Audit, Regulatory Regime, and Whistleblowers policy.
5.  **Four Pillars of Good Governance
Practice:**
    *   **Vision & Mission:** Clear direction, viability, and sustainability.
    *
**Stakeholder Engagement:** Understanding and interacting with key stakeholders.
    *
**Compliance & Management:** Culture of compliance, robust risk management, and performance
management.
    *   **Professional Development & Succession:** Ongoing board development and renewal
planning.
6.  **The "4 P's" of Corporate Governance Achievement:** People, Purpose, Process, and
Performance – outlining key aspects for successful implementation.
7.  **Corporate Governance
Structure:** Directors hold fiduciary positions, overseeing management (CEO and team). Emphasizes
board effectiveness (composition, skills, training, evaluation), shareholder management (engagement,
grievance policies), audit quality, stakeholder engagement, and executive remuneration.
8.
**Environmental, Social, and Governance (ESG):**
    *   A business review of a company’s economic,
environmental, and social impacts.
    *   **Business Responsibility and Sustainability Report
(BRSR):** A significant evolution from BRR, mandatory for top 1000 listed entities from FY22-23,
requiring detailed disclosures on ESG parameters based on NGRBC principles (essential and leadership
indicators).
9.  **Corporate Governance Scorecard:** Developed by BSE in collaboration with IFC and
IiAS, it's a tool to evaluate and benchmark Indian corporates' governance practices based on
publicly available data, aligning with G20/OECD principles.
10. **Climate-Related Financial
Disclosures (TCFD):** Recommendations focusing on four pillars: Governance, Strategy, Risk
Management, and Metrics & Targets, indicating the increasing integration of climate change
considerations into financial reporting and corporate strategy.
11. **Importance and Impact:** CG is
crucial for protecting investor interests, promoting transparency, building public confidence,
ensuring ethical business practices, and creating long-term value for all stakeholders. The e-book
also references historical corporate governance failures in India to underscore its critical need.